# Overfitting a small encoder-only classifier to a dataset. Not using a validation set since i'm mostly just interested in getting the transformer architecture training and not its actual performance or how well it generalizes on this problem

In [18]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ratings/Womens_Clothing_E-Commerce_Reviews_CLEANED.csv


In [19]:
%rm -r transformer_from_scratch
!git clone https://github.com/culv/transformer_from_scratch.git
%cd transformer_from_scratch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


rm: cannot remove 'transformer_from_scratch': No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'transformer_from_scratch'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 72 (delta 37), reused 55 (delta 23), pack-reused 0
Unpacking objects: 100% (72/72), 91.76 KiB | 789.00 KiB/s, done.
/kaggle/working/transformer_from_scratch/transformer_from_scratch


In [20]:
from data_utils import Gpt2Tokenizer

t = Gpt2Tokenizer
print(len(t))
print(t.vocab_size)
print(t.eos_token)
t.pad_token = t.eos_token
print(t.pad_token)

50257
50257
<|endoftext|>
<|endoftext|>


In [21]:
from transformer_from_scratch.data_utils import BalancedClassRandomSampler, PandasDataset, tokenize

data_path = "/kaggle/input/ratings/Womens_Clothing_E-Commerce_Reviews_CLEANED.csv"
ds = PandasDataset(
    data_path,
    input_column="review",
    output_column="Recommended IND",
    input_transform=tokenize(160)
)
print(ds[111])

((tensor([ 7279,  1324,  1563,   278,  3081,   383,  1486,    14, 43358,   286,
          262,  6576,   389,  2407, 48259,    11,   781,  5893,   290, 23564,
           13,   475,  1106,   612,   318,   645,   835,   326,   262,  6576,
         1312,  2722,   318,   649,    13,   262,  3124,   318,   257, 24887,
        18989,   503,  2266,   290,   612,   389,  2042, 43329,   477,   625,
          262, 10999,  1989,    13,   612,   318,   645,  7621,   986,   262,
         9664,  3073,  3102, 11081,   290, 22138,  1068,   290,   318,   407,
        23453,    11, 15175,   393,   649,    13,  1312,   716,   845, 11679,
          416,   262,  3081,   286,   262,  2378,   326,  1312,  2722,    13,
        17713,   428,   530,   318,  1016,   736,    13,   198,   198,    67,
          451, 21538,   532,  3387,   787,  1654,   326,   345,   466,   407,
         3758,   662,    12,   322,   437,  9528,  6685,   284, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [22]:
from torch.utils.data import DataLoader

balanced_sampler = BalancedClassRandomSampler(ds.df[ds.output_column])

dl = DataLoader(ds, batch_size=64, sampler=balanced_sampler)#, shuffle=True)
for (x, m), y in dl:
    print(m)
    print(m.shape)
    print(y)
    print(y.shape)
    break

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
torch.Size([64, 160])
tensor([0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0])
torch.Size([64])


In [23]:
import torch
import torch.nn as nn

from models import EncoderOnlyClassifier

num_heads = 8
d_model = 256
seq_len = 160

model = EncoderOnlyClassifier(
    num_classes=2,
    num_encoders = 6,
    num_heads = 8,
    d_model = d_model,
    seq_len = seq_len
)

loss_fn = nn.CrossEntropyLoss()
lr = 0.00025  # a pretty low learning rate seems to work best
opt = torch.optim.Adam(model.parameters(), lr=lr)

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(f"{device=}")
model.to(device=device)

log_loss_every = 100

epochs = 5
i = 0
for epoch in range(epochs):
    for (x, m), y in dl:
        
        x = x.to(device=device)
        m = m.to(device=device)
        m = m.unsqueeze(-1) * m.unsqueeze(-2)
        y = y.to(device=device)
        
        opt.zero_grad()

        out = model(x, m)
                
        loss = loss_fn(out, y)
        
        loss.backward()
                
        opt.step()

        if i % log_loss_every == 0:
            y_pred = out.argmax(dim=-1)
            
            pos_ids = torch.nonzero(y)
            neg_ids = torch.nonzero(1-y)

            true_pos = (y_pred[pos_ids] == 1).float().sum()
            false_pos = (y_pred[neg_ids] == 1).float().sum()
            true_neg = (y_pred[neg_ids] == 0).float().sum()
            false_neg = (y_pred[pos_ids] == 0).float().sum()

            precision = true_pos / (true_pos + false_pos)
            recall = true_pos / (true_pos + false_neg)
            accuracy = (true_pos + true_neg) / len(y)

            print(
                f"epoch={epoch}, iter={i}, loss={loss.detach().cpu():.4f}, "
                f"accuracy={accuracy.detach().cpu():.4f}, precision={precision.detach().cpu():.4f}, "
                f"recall={recall.detach().cpu():.4f}"
            )
        i += 1

device=device(type='cuda')
epoch=0, iter=0, loss=0.7688, accuracy=0.4688, precision=0.5333, recall=0.2286
epoch=0, iter=100, loss=0.6041, accuracy=0.6562, precision=0.6500, recall=0.7647
epoch=0, iter=200, loss=0.4570, accuracy=0.8125, precision=1.0000, recall=0.6129
epoch=0, iter=300, loss=0.2784, accuracy=0.8906, precision=0.8605, recall=0.9737
epoch=1, iter=400, loss=0.2314, accuracy=0.9375, precision=0.9688, recall=0.9118
epoch=1, iter=500, loss=0.2900, accuracy=0.8906, precision=0.9286, recall=0.8387
epoch=1, iter=600, loss=0.4496, accuracy=0.8281, precision=0.8649, recall=0.8421
epoch=1, iter=700, loss=0.1431, accuracy=0.9219, precision=0.9643, recall=0.8710
epoch=2, iter=800, loss=0.3522, accuracy=0.8594, precision=0.7812, recall=0.9259
epoch=2, iter=900, loss=0.3064, accuracy=0.8438, precision=1.0000, recall=0.6667
epoch=2, iter=1000, loss=0.0802, accuracy=0.9844, precision=1.0000, recall=0.9677
epoch=3, iter=1100, loss=0.0670, accuracy=0.9688, precision=1.0000, recall=0.9394
e

In [25]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

# Moving stuff to CPU so we dont have to do it each time we adjust the visualization sliders
x_ = x.cpu()
m_ = m.cpu()
attns = []
for enc_layer in model.encoder.encoder_layers:
    layer_attns = []
    for head in enc_layer.multihead_self_attention.attentions:
        layer_attns.append(head.detach().cpu())
    attns.append(layer_attns)

def visualize_attention(batch_element, attention_head, encoder_layer):
    inp = t.convert_ids_to_tokens(x_[batch_element])
    lim = (x_[batch_element] == 50256).to(int).argmax().item()

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.matshow(
        attns[encoder_layer][attention_head][batch_element].detach()[:lim,:lim]
    )
    ax.set_yticks(np.arange(0, lim));#seq_len));
    ax.set_yticklabels(inp[:lim]);
    ax.set_xticks(np.arange(0, lim));#seq_len));
    ax.set_xticklabels(inp[:lim], rotation="vertical");

interact(visualize_attention, attention_head=(0,7,1), batch_element=(0,x_.shape[0]-1,1), encoder_layer=(0,5,1))

interactive(children=(IntSlider(value=24, description='batch_element', max=49), IntSlider(value=3, description…

<function __main__.visualize_attention(batch_element, attention_head, encoder_layer)>

In [26]:
def visualize_mask(batch_element):
    m_cpu = m.cpu().detach()
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.matshow(m_cpu[batch_element])

interact(visualize_mask, batch_element=(0,m.shape[0]-1,1))

interactive(children=(IntSlider(value=24, description='batch_element', max=49), Output()), _dom_classes=('widg…

<function __main__.visualize_mask(batch_element)>